In [7]:
pip install snscrape pandas


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import requests
import pandas as pd
from datetime import datetime
import time

def scrape_multiple_pages(subreddit="CryptoCurrency", pages=10, delay=2):
    headers = {"User-Agent": "Mozilla/5.0"}
    after = None
    all_posts = []

    for page in range(pages):
        url = f"https://www.reddit.com/r/{subreddit}/.json"
        if after:
            url += f"?after={after}"

        print(f"📦 Fetching page {page + 1}...")
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print("❌ Failed at page", page + 1, "| Status:", response.status_code)
            break

        data = response.json()
        posts = data["data"]["children"]
        after = data["data"].get("after", None)

        for post in posts:
            p = post["data"]
            all_posts.append({
                "title": p.get("title", ""),
                "author": p.get("author", ""),
                "created_utc": datetime.utcfromtimestamp(p.get("created_utc", 0)).strftime('%Y-%m-%d %H:%M:%S'),
                "score": p.get("score", 0),
                "num_comments": p.get("num_comments", 0),
                "url": p.get("url", "")
            })

        if not after:
            print("🚫 No more pages.")
            break

        time.sleep(delay)  # Be polite to Reddit

    df = pd.DataFrame(all_posts)
    return df

# Run it
if __name__ == "__main__":
    df = scrape_multiple_pages(pages=10)
    df.to_csv("r_cryptocurrency_multi.csv", index=False)
    print(f"✅ Collected {len(df)} posts total")
    print(df.head())

📦 Fetching page 1...


/var/folders/14/lwdqxjmj54dcx0n3tktty0gw0000gn/T/ipykernel_43181/804874314.py:32: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "created_utc": datetime.utcfromtimestamp(p.get("created_utc", 0)).strftime('%Y-%m-%d %H:%M:%S'),


📦 Fetching page 2...
📦 Fetching page 3...
📦 Fetching page 4...
📦 Fetching page 5...
📦 Fetching page 6...
📦 Fetching page 7...
📦 Fetching page 8...
📦 Fetching page 9...
📦 Fetching page 10...
✅ Collected 252 posts total
                                               title                author  \
0  Trade Without Trade-Offs on Kraken Pro! + (Moo...        krakenexchange   
1    Daily Crypto Discussion - April 8, 2025 (GMT+0)          CryptoDaily-   
2  U.S government suffers 26% loss on crypto sinc...               Abdeliq   
3  Just over two month ago Eric Trump was shillin...              GabeSter   
4  Dave Ramsey Predicted Bitcoin Would Crash to $...  InclineDumbbellPress   

           created_utc  score  num_comments  \
0  2025-04-04 14:05:29     22            66   
1  2025-04-08 00:00:47      8           134   
2  2025-04-07 10:57:17   2293           304   
3  2025-04-07 13:24:11   1011            89   
4  2025-04-08 00:38:36    110            35   

                              

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chrome_options = Options()
# chrome_options.add_argument("--headless")  # Debug visible
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0)")

webdriver_service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

def scrape_nitter(query="crypto OR bitcoin OR ethereum", max_scrolls=10, delay=3):
    base_url = "https://nitter.net/search"
    driver.get(f"{base_url}?f=tweets&q={query}")
    tweets_data = set()

    last_height = driver.execute_script("return document.body.scrollHeight")
    for scroll in range(max_scrolls):
        print(f"📜 Scrolling... {scroll + 1}")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("🚫 No more content to scroll.")
            break
        last_height = new_height

        tweets = driver.find_elements(By.CLASS_NAME, "timeline-item")

        for tweet in tweets:
            try:
                username = tweet.find_element(By.CLASS_NAME, "username").text
                date = tweet.find_element(By.CSS_SELECTOR, "span.tweet-date a").text
                tweet_url = tweet.find_element(By.CSS_SELECTOR, "span.tweet-date a").get_attribute("href")
                text = tweet.find_element(By.CLASS_NAME, "tweet-content").text

                tweet_id = f"{username}|{date}|{text[:20]}"  # Unique-ish key
                if tweet_id not in tweets_data:
                    tweets_data.add(tweet_id)
                    yield {
                        "username": username,
                        "date": date,
                        "text": text,
                        "url": tweet_url if "nitter.net" in tweet_url else f"https://nitter.net{tweet_url}"
                    }
            except Exception as e:
                print("⚠️ Skipping tweet:", e)

if __name__ == "__main__":
    collected = list(scrape_nitter(max_scrolls=20))
    df = pd.DataFrame(collected)
    df.to_csv("nitter_crypto_broad.csv", index=False)
    print(f"✅ Saved {len(df)} tweets")
    print(df.head())

    driver.quit()

📜 Scrolling... 1
📜 Scrolling... 2
🚫 No more content to scroll.
✅ Saved 5 tweets
           username    date  \
0    @Cointelegraph      1h   
1          @GENIC0N   Apr 3   
2     @JhonBismarck      6h   
3           @farokh  Jan 10   
4  @AirdropNinjaPro   Apr 5   

                                                text  \
0  ⚡NEW: A trader got liquidated for $100M yester...   
1  Anime cat girls will be mass manufactured in u...   
2  AVM will revolutionize the Bitcoin ecosystem, ...   
3  I attended the opening of the new @Ledger HQ i...   
4  New Airdrop: SALTY\nReward: 150,000 SALTY \nLi...   

                                                 url  
0  https://nitter.net/Cointelegraph/status/190944...  
1  https://nitter.net/GENIC0N/status/190762753788...  
2  https://nitter.net/JhonBismarck/status/1909378...  
3  https://nitter.net/farokh/status/1877716958025...  
4  https://nitter.net/AirdropNinjaPro/status/1908...  


In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

# 🔧 Set up headless Chrome
options = Options()
options.add_argument("--headless")  # Remove if you want to see browser
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0")

# ✅ Update this to your ChromeDriver path
driver_path = "/opt/homebrew/bin/chromedriver"  # Change if needed
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

def scrape_coindesk_headlines():
    driver.get("https://www.coindesk.com/")
    time.sleep(5)  # Give time for JS to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    headlines = []

    # Find all anchor tags with title attribute and class indicating headline
    for a in soup.find_all("a", attrs={"title": True, "href": True}):
        title = a["title"].strip()
        href = a["href"].strip()
        if href.startswith("/"):  # relative path
            full_url = "https://www.coindesk.com" + href
        else:
            full_url = href

        headlines.append({
            "headline": title,
            "url": full_url,
            "scraped_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })

    return pd.DataFrame(headlines)

if __name__ == "__main__":
    df = scrape_coindesk_headlines()
    driver.quit()

    df.drop_duplicates(subset="url", inplace=True)
    df.to_csv("coindesk_latest_news.csv", index=False)
    print(f"✅ Scraped {len(df)} headlines from CoinDesk.")
    print(df.head())

✅ Scraped 52 headlines from CoinDesk.
             headline                                          url  \
0  View price details       https://www.coindesk.com/price/bitcoin   
1  View price details      https://www.coindesk.com/price/ethereum   
2  View price details        https://www.coindesk.com/price/tether   
3  View price details           https://www.coindesk.com/price/xrp   
4  View price details  https://www.coindesk.com/price/binance-coin   

            scraped_at  
0  2025-04-08 00:08:49  
1  2025-04-08 00:08:49  
2  2025-04-08 00:08:49  
3  2025-04-08 00:08:49  
4  2025-04-08 00:08:49  


In [35]:
import requests
import pandas as pd
from datetime import datetime
import time

def scrape_reddit(subreddit="CryptoCurrency", pages=20, delay=2):
    headers = {"User-Agent": "Mozilla/5.0"}
    after = None
    posts = []

    for page in range(pages):
        url = f"https://www.reddit.com/r/{subreddit}/.json"
        if after:
            url += f"?after={after}"

        print(f"📦 Fetching r/{subreddit} page {page + 1}")
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"❌ Failed at page {page + 1} | Status:", response.status_code)
            break

        data = response.json()
        children = data["data"]["children"]
        after = data["data"].get("after", None)

        for post in children:
            p = post["data"]
            posts.append({
                "subreddit": subreddit,
                "title": p.get("title", ""),
                "author": p.get("author", ""),
                "created_utc": datetime.utcfromtimestamp(p.get("created_utc", 0)).strftime('%Y-%m-%d %H:%M:%S'),
                "score": p.get("score", 0),
                "num_comments": p.get("num_comments", 0),
                "url": p.get("url", "")
            })

        if not after:
            print(f"🚫 No more pages for r/{subreddit}")
            break

        time.sleep(delay)

    return posts

# 🔁 Combine from multiple subreddits
if __name__ == "__main__":
    all_subreddits = ["CryptoCurrency", "Bitcoin", "CryptoMarkets", "CryptoTechnology", "CryptoNews"]
    all_posts = []

    for sub in all_subreddits:
        posts = scrape_reddit(subreddit=sub, pages=20)
        all_posts.extend(posts)

    df = pd.DataFrame(all_posts)
    df.drop_duplicates(subset=["title", "url"], inplace=True)
    df.to_csv("reddit_crypto_bulk.csv", index=False)

    print(f"✅ Collected {len(df)} total posts across {len(all_subreddits)} subreddits.")
    print(df.head())

📦 Fetching r/CryptoCurrency page 1


/var/folders/14/lwdqxjmj54dcx0n3tktty0gw0000gn/T/ipykernel_43181/2327192113.py:32: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "created_utc": datetime.utcfromtimestamp(p.get("created_utc", 0)).strftime('%Y-%m-%d %H:%M:%S'),


📦 Fetching r/CryptoCurrency page 2
📦 Fetching r/CryptoCurrency page 3
📦 Fetching r/CryptoCurrency page 4
📦 Fetching r/CryptoCurrency page 5
📦 Fetching r/CryptoCurrency page 6
📦 Fetching r/CryptoCurrency page 7
📦 Fetching r/CryptoCurrency page 8
📦 Fetching r/CryptoCurrency page 9
📦 Fetching r/CryptoCurrency page 10
📦 Fetching r/CryptoCurrency page 11
📦 Fetching r/CryptoCurrency page 12
📦 Fetching r/CryptoCurrency page 13
📦 Fetching r/CryptoCurrency page 14
📦 Fetching r/CryptoCurrency page 15
🚫 No more pages for r/CryptoCurrency
📦 Fetching r/Bitcoin page 1
📦 Fetching r/Bitcoin page 2
📦 Fetching r/Bitcoin page 3
📦 Fetching r/Bitcoin page 4
📦 Fetching r/Bitcoin page 5
📦 Fetching r/Bitcoin page 6
📦 Fetching r/Bitcoin page 7
📦 Fetching r/Bitcoin page 8
📦 Fetching r/Bitcoin page 9
📦 Fetching r/Bitcoin page 10
📦 Fetching r/Bitcoin page 11
📦 Fetching r/Bitcoin page 12
📦 Fetching r/Bitcoin page 13
📦 Fetching r/Bitcoin page 14
📦 Fetching r/Bitcoin page 15
📦 Fetching r/Bitcoin page 16
📦 Fetching r

In [36]:
import requests
import pandas as pd
from datetime import datetime
import time

def scrape_reddit_bulk(subreddits, pages_per_sub=20, delay=2):
    headers = {"User-Agent": "Mozilla/5.0"}
    all_posts = []

    for subreddit in subreddits:
        after = None
        print(f"\n🚀 Starting r/{subreddit}...")
        for page in range(pages_per_sub):
            url = f"https://www.reddit.com/r/{subreddit}/.json"
            if after:
                url += f"?after={after}"

            print(f"📦 Fetching page {page + 1} of r/{subreddit}")
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"❌ Failed at page {page + 1} | Status Code:", response.status_code)
                break

            data = response.json()
            posts = data["data"]["children"]
            after = data["data"].get("after", None)

            for post in posts:
                p = post["data"]
                all_posts.append({
                    "subreddit": subreddit,
                    "title": p.get("title", ""),
                    "author": p.get("author", ""),
                    "created_utc": datetime.utcfromtimestamp(p.get("created_utc", 0)).strftime('%Y-%m-%d %H:%M:%S'),
                    "score": p.get("score", 0),
                    "num_comments": p.get("num_comments", 0),
                    "url": p.get("url", "")
                })

            if not after:
                print("🚫 No more pages.")
                break
            time.sleep(delay)

    return pd.DataFrame(all_posts)

# 🔁 Run the collector
if __name__ == "__main__":
    target_subreddits = [
        "CryptoCurrency",
        "Bitcoin",
        "CryptoMarkets",
        "CryptoTechnology",
        "CryptoNews"
    ]

    df = scrape_reddit_bulk(target_subreddits, pages_per_sub=20)  # 20 pages x ~25 posts x 5 subs ≈ 2500+
    df.drop_duplicates(subset=["title", "url"], inplace=True)
    df.to_csv("reddit_crypto_2000_posts.csv", index=False)

    print(f"\n✅ DONE: Collected {len(df)} Reddit posts total across {len(target_subreddits)} subs.")
    print(df.sample(5))


🚀 Starting r/CryptoCurrency...
📦 Fetching page 1 of r/CryptoCurrency


/var/folders/14/lwdqxjmj54dcx0n3tktty0gw0000gn/T/ipykernel_43181/2401029582.py:34: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "created_utc": datetime.utcfromtimestamp(p.get("created_utc", 0)).strftime('%Y-%m-%d %H:%M:%S'),


📦 Fetching page 2 of r/CryptoCurrency
📦 Fetching page 3 of r/CryptoCurrency
📦 Fetching page 4 of r/CryptoCurrency
📦 Fetching page 5 of r/CryptoCurrency
📦 Fetching page 6 of r/CryptoCurrency
📦 Fetching page 7 of r/CryptoCurrency
📦 Fetching page 8 of r/CryptoCurrency
📦 Fetching page 9 of r/CryptoCurrency
📦 Fetching page 10 of r/CryptoCurrency
📦 Fetching page 11 of r/CryptoCurrency
📦 Fetching page 12 of r/CryptoCurrency
📦 Fetching page 13 of r/CryptoCurrency
📦 Fetching page 14 of r/CryptoCurrency
📦 Fetching page 15 of r/CryptoCurrency
🚫 No more pages.

🚀 Starting r/Bitcoin...
📦 Fetching page 1 of r/Bitcoin
📦 Fetching page 2 of r/Bitcoin
📦 Fetching page 3 of r/Bitcoin
📦 Fetching page 4 of r/Bitcoin
📦 Fetching page 5 of r/Bitcoin
📦 Fetching page 6 of r/Bitcoin
📦 Fetching page 7 of r/Bitcoin
📦 Fetching page 8 of r/Bitcoin
📦 Fetching page 9 of r/Bitcoin
📦 Fetching page 10 of r/Bitcoin
📦 Fetching page 11 of r/Bitcoin
📦 Fetching page 12 of r/Bitcoin
📦 Fetching page 13 of r/Bitcoin
📦 Fetching pa

In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

def scroll_to_bottom(driver, scrolls=100, delay=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)
        new_height = driver.execute_script("return document.body.scrollHeight")
        print(f"📜 Scroll {i+1}/{scrolls}")
        if new_height == last_height:
            print("🚫 Reached end of page.")
            break
        last_height = new_height

def scrape_cointelegraph_bitcoin_articles():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0")

    driver_path = "/opt/homebrew/bin/chromedriver"  # Update path if needed
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=options)

    url = "https://cointelegraph.com/tags/bitcoin"
    print(f"🌐 Opening: {url}")
    driver.get(url)
    time.sleep(5)

    scroll_to_bottom(driver, scrolls=150, delay=2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    links = soup.select("a.post-card-inline__title-link")

    articles = []
    for a in links:
        headline = a.text.strip()
        href = a.get("href", "")
        if headline and href:
            full_url = "https://cointelegraph.com" + href if href.startswith("/") else href
            articles.append({
                "source": "CoinTelegraph",
                "tag": "bitcoin",
                "headline": headline,
                "url": full_url,
                "scraped_at": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
            })

    driver.quit()
    return pd.DataFrame(articles)

# Run it
if __name__ == "__main__":
    df = scrape_cointelegraph_bitcoin_articles()
    df.drop_duplicates(subset="url", inplace=True)
    df.to_excel("cointelegraph_bitcoin_scroll_fixed.xlsx", index=False)

    print(f"\n✅ Scraped {len(df)} Bitcoin articles from CoinTelegraph")
    print(df.sample(min(5, len(df))))

🌐 Opening: https://cointelegraph.com/tags/bitcoin
📜 Scroll 1/150
📜 Scroll 2/150
📜 Scroll 3/150
📜 Scroll 4/150
📜 Scroll 5/150
📜 Scroll 6/150
📜 Scroll 7/150
🚫 Reached end of page.

✅ Scraped 105 Bitcoin articles from CoinTelegraph
           source      tag                                           headline  \
79  CoinTelegraph  bitcoin  GameStop finishes $1.5B raise to add Bitcoin t...   
46  CoinTelegraph  bitcoin  Trump tariffs squeeze already struggling Bitco...   
86  CoinTelegraph  bitcoin  Bitcoin mining using coal energy down 43% sinc...   
29  CoinTelegraph  bitcoin  No country wins a global trade war, BTC to sur...   
4   CoinTelegraph  bitcoin               Here’s what happened in crypto today   

                                                  url           scraped_at  
79  https://cointelegraph.com/news/game-stop-finis...  2025-04-08 06:48:16  
46  https://cointelegraph.com/news/trump-tariffs-b...  2025-04-08 06:48:16  
86  https://cointelegraph.com/news/bitcoin-mining-...

/var/folders/14/lwdqxjmj54dcx0n3tktty0gw0000gn/T/ipykernel_43181/1473940868.py:53: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "scraped_at": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")


In [58]:
import requests
import pandas as pd
from datetime import datetime
import time

def scrape_cryptoslate_wpapi(pages=10):
    base_url = "https://cryptoslate.com/wp-json/wp/v2/posts?page={}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    all_articles = []

    for page in range(1, pages + 1):
        url = base_url.format(page)
        print(f"🔄 Fetching page {page}: {url}")
        resp = requests.get(url, headers=headers)

        if resp.status_code != 200:
            print(f"❌ Failed at page {page} | Status {resp.status_code}")
            break

        data = resp.json()
        if not data:
            print("🚫 No data returned.")
            break

        for item in data:
            all_articles.append({
                "title": item.get("title", {}).get("rendered", ""),
                "link": item.get("link", ""),
                "date": item.get("date", ""),
                "scraped_at": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
            })

        time.sleep(1)

    return pd.DataFrame(all_articles)

# ✅ Run it
if __name__ == "__main__":
    df = scrape_cryptoslate_wpapi(pages=25)
    df.to_excel("cryptoslate_articles_wpapi.xlsx", index=False)
    print(f"✅ Saved {len(df)} articles to cryptoslate_articles_wpapi.xlsx")
    print(df.sample(5))

🔄 Fetching page 1: https://cryptoslate.com/wp-json/wp/v2/posts?page=1


/var/folders/14/lwdqxjmj54dcx0n3tktty0gw0000gn/T/ipykernel_43181/3652343853.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "scraped_at": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")


🔄 Fetching page 2: https://cryptoslate.com/wp-json/wp/v2/posts?page=2
🔄 Fetching page 3: https://cryptoslate.com/wp-json/wp/v2/posts?page=3
🔄 Fetching page 4: https://cryptoslate.com/wp-json/wp/v2/posts?page=4
🔄 Fetching page 5: https://cryptoslate.com/wp-json/wp/v2/posts?page=5
🔄 Fetching page 6: https://cryptoslate.com/wp-json/wp/v2/posts?page=6
🔄 Fetching page 7: https://cryptoslate.com/wp-json/wp/v2/posts?page=7
🔄 Fetching page 8: https://cryptoslate.com/wp-json/wp/v2/posts?page=8
🔄 Fetching page 9: https://cryptoslate.com/wp-json/wp/v2/posts?page=9
🔄 Fetching page 10: https://cryptoslate.com/wp-json/wp/v2/posts?page=10
🔄 Fetching page 11: https://cryptoslate.com/wp-json/wp/v2/posts?page=11
🔄 Fetching page 12: https://cryptoslate.com/wp-json/wp/v2/posts?page=12
🔄 Fetching page 13: https://cryptoslate.com/wp-json/wp/v2/posts?page=13
🔄 Fetching page 14: https://cryptoslate.com/wp-json/wp/v2/posts?page=14
🔄 Fetching page 15: https://cryptoslate.com/wp-json/wp/v2/posts?page=15
🔄 Fetchi

In [71]:
pip install httpx


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "httpx[http2]"


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.0/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup Chrome driver
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0")

driver_path = "/opt/homebrew/bin/chromedriver"  # Change if necessary
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

def scrape_coinspeaker_news(short_scrolls=4, delay=3):
    print("🌐 Opening CoinSpeaker News Page...")
    driver.get("https://www.coinspeaker.com/news/")
    articles = []

    for i in range(short_scrolls):
        print(f"📜 Scroll {i + 1}/{short_scrolls}")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)

    # Wait for at least 1 article to show up
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "news-flow-card"))
        )
    except Exception as e:
        print("❌ Timed out or no articles found.")
        driver.quit()
        return pd.DataFrame()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    cards = soup.find_all("div", class_="news-flow-card")

    for card in cards:
        a_tag = card.find("a", class_="title")
        if a_tag:
            title = a_tag.get_text(strip=True)
            href = a_tag.get("href")
            url = href if href.startswith("http") else f"https://www.coinspeaker.com{href}"
            articles.append({"title": title, "url": url})

    driver.quit()
    return pd.DataFrame(articles)

# Run it for testing 4 scrolls
df = scrape_coinspeaker_news(short_scrolls=4)
df.drop_duplicates(subset="url", inplace=True)
df.to_csv("coinspeaker_short_scroll_news.csv", index=False)
df.to_excel("coinspeaker_short_scroll_news.xlsx", index=False)

print(f"\n✅ Scraped {len(df)} articles from CoinSpeaker (4 scrolls).")
print(df.head())

🌐 Opening CoinSpeaker News Page...
📜 Scroll 1/4
📜 Scroll 2/4
📜 Scroll 3/4
📜 Scroll 4/4
❌ Timed out or no articles found.

✅ Scraped 0 articles from CoinSpeaker (4 scrolls).
Empty DataFrame
Columns: []
Index: []
